In [2]:
pr_df = spark.read.option("header","true").format("csv").load("{path_to_the_file}/diabetic_data_workshop.csv", inferSchema="true")
pr_df.count()



22362

In [3]:
pr_df.printSchema()

root
 |-- race: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: string (nullable = true)
 |-- admission_type_id: integer (nullable = true)
 |-- discharge_disposition_id: integer (nullable = true)
 |-- admission_source_id: integer (nullable = true)
 |-- time_in_hospital: integer (nullable = true)
 |-- payer_code: string (nullable = true)
 |-- medical_specialty: string (nullable = true)
 |-- num_lab_procedures: integer (nullable = true)
 |-- num_procedures: integer (nullable = true)
 |-- num_medications: integer (nullable = true)
 |-- number_outpatient: integer (nullable = true)
 |-- number_emergency: integer (nullable = true)
 |-- number_inpatient: integer (nullable = true)
 |-- diag_1: string (nullable = true)
 |-- diag_2: string (nullable = true)
 |-- diag_3: string (nullable = true)
 |-- number_diagnoses: integer (nullable = true)
 |-- max_glu_serum: integer (nullable = true)
 |-- A1Cresult: integer (nullable = true)
 |-- metformin: string (nullable = true)
 

In [4]:
# print(type(pr_df))
pr_df.show(2,truncate= True)

+---------------+------+-------+-----------------+------------------------+-------------------+----------------+----------+--------------------+------------------+--------------+---------------+-----------------+----------------+----------------+-----------+------------------+------------------+----------------+-------------+---------+---------+-----------+---------+-------+------+-----------+----------+-----------+
|           race|gender|    age|admission_type_id|discharge_disposition_id|admission_source_id|time_in_hospital|payer_code|   medical_specialty|num_lab_procedures|num_procedures|num_medications|number_outpatient|number_emergency|number_inpatient|     diag_1|            diag_2|            diag_3|number_diagnoses|max_glu_serum|A1Cresult|metformin|repaglinide|glipizide|insulin|change|diabetesMed|readmitted|first_visit|
+---------------+------+-------+-----------------+------------------------+-------------------+----------------+----------+--------------------+----------------

In [5]:
pr_df.describe().show()

+-------+---------------+---------------+--------+------------------+------------------------+-------------------+------------------+----------+--------------------+------------------+------------------+------------------+-------------------+------------------+------------------+--------------------+--------------------+--------------------+------------------+-------------------+-------------------+---------+-----------+---------+-------+------+-----------+------------------+------------------+
|summary|           race|         gender|     age| admission_type_id|discharge_disposition_id|admission_source_id|  time_in_hospital|payer_code|   medical_specialty|num_lab_procedures|    num_procedures|   num_medications|  number_outpatient|  number_emergency|  number_inpatient|              diag_1|              diag_2|              diag_3|  number_diagnoses|      max_glu_serum|          A1Cresult|metformin|repaglinide|glipizide|insulin|change|diabetesMed|        readmitted|       first_visit|


In [7]:
import matplotlib.pyplot as plt
# Show histogram of the 'time_in_hospital' column
bins, counts = pr_df.select('time_in_hospital').rdd.flatMap(lambda x: x).histogram(20)

plt.hist(bins[:-1], bins=bins, weights=counts)

(array([2604., 3492.,    0., 3624., 3150.,    0., 2304., 1789.,    0.,
        1374., 1146.,    0.,  802.,  642.,    0.,  444.,  378.,    0.,
         319.,  294.]),
 array([ 1.  ,  1.65,  2.3 ,  2.95,  3.6 ,  4.25,  4.9 ,  5.55,  6.2 ,
         6.85,  7.5 ,  8.15,  8.8 ,  9.45, 10.1 , 10.75, 11.4 , 12.05,
        12.7 , 13.35, 14.  ]),
 <a list of 20 Patch objects>)

In [8]:
# Transformers - Transform

from pyspark.ml.feature import Binarizer
from pyspark.sql.types import DoubleType

pr_df = pr_df.withColumn("time_in_hospital_double", pr_df["time_in_hospital"].cast(DoubleType()))

time_in_hospital_binarizer = Binarizer(threshold=4, inputCol="time_in_hospital_double", outputCol='time_in_hospital_binarized' )
pr_df = time_in_hospital_binarizer.transform(pr_df)
pr_df.select("time_in_hospital","time_in_hospital_double", "time_in_hospital_binarized").show()



+----------------+-----------------------+--------------------------+
|time_in_hospital|time_in_hospital_double|time_in_hospital_binarized|
+----------------+-----------------------+--------------------------+
|               7|                    7.0|                       1.0|
|               7|                    7.0|                       1.0|
|               4|                    4.0|                       0.0|
|               9|                    9.0|                       1.0|
|               4|                    4.0|                       0.0|
|               2|                    2.0|                       0.0|
|               7|                    7.0|                       1.0|
|               3|                    3.0|                       0.0|
|              14|                   14.0|                       1.0|
|               4|                    4.0|                       0.0|
|              10|                   10.0|                       1.0|
|               1|  

In [9]:
pr_df.select("gender").distinct().show()
# a = pr_df.groupBy("gender").count()
# a.show()


+---------------+
|         gender|
+---------------+
|         Female|
|Unknown/Invalid|
|           Male|
+---------------+



In [10]:
pr_df.crosstab('gender', 'readmitted').show()

+-----------------+----+----+
|gender_readmitted|   0|   1|
+-----------------+----+----+
|  Unknown/Invalid|   0|   4|
|             Male|5225|5137|
|           Female|5974|6022|
+-----------------+----+----+



In [11]:
# Estimators - Train
# .fit to train
from pyspark.ml.feature import StringIndexer

gender_indexer = StringIndexer(inputCol = "gender", outputCol="gender_index")
gender_uniqueDf = pr_df.select("gender").distinct()
gender_uniqueDf.show()


gender_indexerModel = gender_indexer.fit(gender_uniqueDf)
gender_indexedDf = gender_indexerModel.transform(gender_uniqueDf)
gender_indexedDf.show()

from pyspark.ml.feature import OneHotEncoderEstimator
gender_encoder = OneHotEncoderEstimator(inputCols=["gender_index"], outputCols=["encoded_gender"])
gender_encoderModel = gender_encoder.fit(gender_indexedDf)
gender_encodedDf = gender_encoderModel.transform(gender_indexedDf)
gender_encodedDf.show()


+---------------+
|         gender|
+---------------+
|         Female|
|Unknown/Invalid|
|           Male|
+---------------+

+---------------+------------+
|         gender|gender_index|
+---------------+------------+
|         Female|         0.0|
|Unknown/Invalid|         1.0|
|           Male|         2.0|
+---------------+------------+

+---------------+------------+--------------+
|         gender|gender_index|encoded_gender|
+---------------+------------+--------------+
|         Female|         0.0| (2,[0],[1.0])|
|Unknown/Invalid|         1.0| (2,[1],[1.0])|
|           Male|         2.0|     (2,[],[])|
+---------------+------------+--------------+



In [12]:
# Encoders

race_indexer = StringIndexer(inputCol = "race", outputCol="race_index")
gender_indexer = StringIndexer(inputCol = "gender", outputCol="gender_index")
payer_code_indexer = StringIndexer(inputCol = "payer_code", outputCol="payer_code_index")
medical_specialty_indexer = StringIndexer(inputCol = "medical_specialty", outputCol="medical_specialty_index")
diag_1_indexer = StringIndexer(inputCol = "diag_1", outputCol="diag_1_index")
diag_2_indexer = StringIndexer(inputCol = "diag_2", outputCol="diag_2_index")
diag_3_indexer = StringIndexer(inputCol = "diag_3", outputCol="diag_3_index")
metformin_indexer = StringIndexer(inputCol = "metformin", outputCol="metformin_index")
repaglinide_indexer = StringIndexer(inputCol = "repaglinide", outputCol="repaglinide_index")
glipizide_indexer = StringIndexer(inputCol = "glipizide", outputCol="glipizide_index")
insulin_indexer = StringIndexer(inputCol = "insulin", outputCol="insulin_index")
change_indexer = StringIndexer(inputCol = "change", outputCol="change_index")
diabetesMed_indexer = StringIndexer(inputCol = "diabetesMed", outputCol="diabetesMed_index")


features_encoder = OneHotEncoderEstimator(inputCols=["race_index", "gender_index", "payer_code_index", "medical_specialty_index", "diag_1_index",  "diag_2_index",  "diag_3_index", "metformin_index", "repaglinide_index", "glipizide_index" , "insulin_index", "change_index", "diabetesMed_index" ], 
                                          outputCols=["encoded_race", "encoded_gender", "encoded_payer_code", "encoded_medical_specialty", "encoded_diag_1", "encoded_diag_2", "encoded_diag_3", "encoded_metformin", "encoded_repaglinide", "encoded_glipizide", "encoded_insulin", "encoded_change", "encoded_diabetesMed" ])

In [13]:
# Feature Columns

featureCols =[
    "admission_type_id",
    "discharge_disposition_id",
    "admission_source_id",
    "time_in_hospital",
    "num_lab_procedures",
    "num_procedures",
    "num_medications",
    "number_outpatient",
    "number_emergency",
    "number_inpatient",
    "number_diagnoses",
    "max_glu_serum",
    "A1Cresult",
    "first_visit",
    "encoded_race",
    "encoded_gender",
    "encoded_payer_code",
    "encoded_medical_specialty",
    "encoded_diag_1",
    "encoded_diag_2",
    "encoded_diag_3",
    "encoded_metformin",
    "encoded_repaglinide",
    "encoded_glipizide",
    "encoded_insulin",
    "encoded_change",
    "encoded_diabetesMed"
]

In [14]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols = featureCols, outputCol="features")

from pyspark.ml.classification import RandomForestClassifier
randomForest = RandomForestClassifier(labelCol="readmitted", featuresCol="features", numTrees=20)

In [15]:
# Split Data 
seed = 42

test_df, train_df = pr_df.randomSplit((0.2, 0.8), seed=seed)
print(test_df.count(), train_df.count())


4497 17865


In [16]:
from pyspark.ml import Pipeline
from pyspark.ml import PipelineModel

featurizationPipeline = Pipeline(stages=[race_indexer,
gender_indexer,
payer_code_indexer,
medical_specialty_indexer,
diag_1_indexer,
diag_2_indexer,
diag_3_indexer,
metformin_indexer,
repaglinide_indexer,
glipizide_indexer,
insulin_indexer,
change_indexer,
diabetesMed_indexer,
features_encoder,
assembler,
randomForest])

model = featurizationPipeline.fit(train_df)
outPath = "/Users/satya/PycharmProjects/datascience_workshop/notebooks/data/rfmodel/"
model.write().overwrite().save(outPath)




In [20]:
loadedModel= PipelineModel.load(outPath)
predictions = loadedModel.transform(test_df)
predictions.select("probability", "prediction", "readmitted", "features").show(10)
predictions.show(3, truncate=True)


+--------------------+----------+----------+--------------------+
|         probability|prediction|readmitted|            features|
+--------------------+----------+----------+--------------------+
|[0.67992495852063...|       0.0|         0|(127,[0,1,2,3,4,6...|
|[0.22590700063095...|       1.0|         1|(127,[0,1,2,3,4,6...|
|[0.72200614507768...|       0.0|         0|(127,[0,1,2,3,4,6...|
|[0.72200614507768...|       0.0|         0|(127,[0,1,2,3,4,6...|
|[0.73719033834538...|       0.0|         0|(127,[0,1,2,3,4,6...|
|[0.73719033834538...|       0.0|         0|(127,[0,1,2,3,4,6...|
|[0.73719033834538...|       0.0|         0|(127,[0,1,2,3,4,6...|
|[0.72200614507768...|       0.0|         0|(127,[0,1,2,3,4,6...|
|[0.73719033834538...|       0.0|         0|(127,[0,1,2,3,4,6...|
|[0.70533022592371...|       0.0|         0|(127,[0,1,2,3,4,6...|
+--------------------+----------+----------+--------------------+
only showing top 10 rows

+---------------+------+-------+-----------------+

In [18]:
# Select (prediction, true label) and compute test error
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(
    labelCol="readmitted", predictionCol="prediction", metricName="accuracy")

accuracy = evaluator.evaluate(predictions)
print("Accuray = %g" % (accuracy))

Accuray = 0.802535


In [19]:
modelSummary = loadedModel.stages[-1]
modelSummary.toDebugString, modelSummary.featureImportances, modelSummary.trees

('RandomForestClassificationModel (uid=RandomForestClassifier_f084fa6689e9) with 20 trees\n  Tree 0 (weight 1.0):\n    If (feature 41 in {1.0})\n     If (feature 13 <= 0.5)\n      If (feature 119 in {0.0})\n       If (feature 21 in {1.0})\n        Predict: 0.0\n       Else (feature 21 not in {1.0})\n        Predict: 1.0\n      Else (feature 119 not in {0.0})\n       If (feature 10 <= 4.5)\n        If (feature 5 <= 0.5)\n         Predict: 0.0\n        Else (feature 5 > 0.5)\n         Predict: 1.0\n       Else (feature 10 > 4.5)\n        Predict: 1.0\n     Else (feature 13 > 0.5)\n      If (feature 1 <= 22.5)\n       Predict: 1.0\n      Else (feature 1 > 22.5)\n       Predict: 0.0\n    Else (feature 41 not in {1.0})\n     If (feature 29 in {1.0})\n      Predict: 1.0\n     Else (feature 29 not in {1.0})\n      If (feature 9 <= 0.5)\n       If (feature 23 in {1.0})\n        Predict: 1.0\n       Else (feature 23 not in {1.0})\n        If (feature 7 <= 0.5)\n         Predict: 0.0\n        El